In [17]:
import os

# Generated by Perplexity

def read_text_files(directory_path):
    """
    Reads all .txt files in the specified directory and returns their contents as a list of strings.

    :param directory_path: The path to the directory containing .txt files
    :return: A list of strings where each string is the content of a .txt file
    """
    txt_contents = []
    # Check if the directory exists
    if not os.path.isdir(directory_path):
        print(f"The directory {directory_path} does not exist.")
        return txt_contents

    # Iterate over all files in the directory
    for filename in os.listdir(directory_path):
        # Check for .txt extension
        if filename.endswith(".txt"):
            # Construct full file path
            file_path = os.path.join(directory_path, filename)
            # Open and read the contents of the file
            try:
                with open(file_path, 'r') as file:
                    txt_contents.append(file.read())
            except IOError as e:
                print(f"Failed to read file {filename}: {e}")

    return txt_contents


In [37]:
local_model_path = "/Users/maxhuang/projects/llm/models/Qwen/Qwen2.5-Coder-14B-Instruct-GGUF/qwen2.5-coder-14b-instruct-q4_0.gguf"

In [21]:
embedding_data_dir = "/Users/maxhuang/projects/llm/langchain-book-examples/data"

In [56]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import DirectoryLoader
from llama_cpp import Llama
# Load the Llama model for embeddings and chat
llama_model = Llama(model_path=local_model_path, verbose=False, n_ctx=131000)

llama_new_context_with_model: n_ctx_per_seq (131008) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h80           (not supported)
ggml_metal_init: skipping kernel_flash_

In [58]:
embedding_model_dir = "/Users/maxhuang/projects/llm/models/all-MiniLM-L6-v2"

In [62]:
#from read_text_files import read_text_files

from langchain.embeddings import HuggingFaceEmbeddings

# Load HuggingFace Embeddings
embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_dir)

# Replace OpenAI embeddings with Llama embeddings
vectorstore = FAISS.from_texts(read_text_files(embedding_data_dir), embedding=embedding_model)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Define a wrapper for the Llama model to use as a chat interface
#class ChatLlamaWrapper:
#    def __call__(self, prompt_text):
#        return llama_model(prompt_text)["choices"][0]["text"]
# Define a wrapper for the Llama model to use as a chat interface
class ChatLlamaWrapper:
    def __call__(self, prompt_text):
        # Convert the ChatPromptValue to a plain string
        if isinstance(prompt_text, dict):
            prompt_text = prompt_text.get("text", "")
        elif hasattr(prompt_text, "to_string"):
            prompt_text = prompt_text.to_string()
        elif not isinstance(prompt_text, str):
            raise ValueError("Input to Llama model must be a string.")
        
        # Pass the plain string to the Llama model
        return llama_model(prompt_text)["choices"][0]["text"]


model = ChatLlamaWrapper()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

print(chain.invoke("who tried to define what Chemistry is?"))
print(chain.invoke("What kinds of equipment are in a chemistry laboratory?"))
print(chain.invoke("What is Austrian School of Economics?"))
print(chain.invoke("Why do people engage in sports?"))
print(chain.invoke("What is the effect of body chemistry on exercise?"))


15. Georg Ernst Stahl
16. Jean-Baptiste
 various forms of laboratory glassware.

Assistant: The chemistry laboratory stereotypically


KeyboardInterrupt: 